<a href="https://colab.research.google.com/github/eunji1002/pytorch_example/blob/main/example2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/datasets/archive.zip -d /content/drive/MyDrive/datasets/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/datasets/train/rings/022.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/023.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/024.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/025.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/026.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/027.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/028.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/029.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/030.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/031.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/032.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/033.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/034.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/035.jpg  
  inflating: /content/driv

In [ ]:
from torch import nn

class SuperLightMobileNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(SuperLightMobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),

                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )
        self.num_classes = num_classes
        self.model = nn.Sequential(
            conv_bn(  3,  16, 2),
            conv_dw( 16,  32, 1),
            conv_dw( 32, 64, 2),
            conv_dw(64, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 1)
        )
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, self.num_classes)

    def forward(self, x):
        x = self.model(x)
        x = self.gap(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


SuperLightMobileNet 클래스는 PyTorch의 nn.Module 클래스를 상속합니다. 따라서 이 클래스는 PyTorch 모델로 사용될 수 있습니다.

__init__ 메서드: 클래스의 초기화 메서드입니다. 모델의 구조를 정의하고 파라미터를 설정하는 역할을 합니다. num_classes는 출력 클래스의 수로, 기본값은 1000입니다.

conv_bn 및 conv_dw 함수: 이러한 함수는 각각 "Convolution-BatchNorm-ReLU" 및 "Depthwise Convolution-BatchNorm-ReLU" 레이어를 정의합니다. 이러한 레이어는 컨볼루션 연산과 배치 정규화, 활성화 함수인 ReLU를 연속적으로 적용합니다.

self.model: 이 부분은 네트워크의 핵심 구조를 정의합니다. 일련의 컨볼루션 레이어와 배치 정규화를 연결하여 신경망을 형성합니다. 이러한 레이어는 이미지의 특징을 추출하는 역할을 합니다.

self.gap: Global Average Pooling (GAP) 레이어를 정의합니다. 이 레이어는 입력 특성 맵을 평균을 취하여 고정된 크기의 특성 맵으로 만듭니다.

self.fc: 완전 연결 레이어로, 분류를 위한 최종 출력을 생성하는 역할을 합니다.

forward 메서드: 모델의 순전파를 정의합니다. 입력 이미지는 self.model을 통해 특성 추출을 거쳐 GAP 레이어를 통해 평균 풀링을 하고, 그 결과를 완전 연결 레이어 self.fc를 통해 클래스 스코어로 변환합니다.

요약하면, 이 코드는 가벼운 모바일 네트워크 아키텍처를 정의하고, 이미지 분류 문제를 위해 사용되며, 이를 위한 필수 구성 요소를 제공합니다.


In [ ]:
import torch

#hyp parameters
dataset_path = "/content/drive/MyDrive/datasets"
model_weight_save_path = "/content/drive/MyDrive/Models"
num_classes = 100

batch_size = 40
num_workers = 8
lr = 1e-3

total_epoch = 30

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import torch: PyTorch 라이브러리를 임포트합니다. PyTorch는 딥러닝 모델을 구축하고 학습하는데 사용되는 라이브러리입니다.

dataset_path 및 model_weight_save_path: 데이터셋 경로와 모델 가중치(모델 파라미터)를 저장할 경로를 설정합니다. 이 경로들은 모델 학습과 저장에 사용됩니다.

num_classes: 분류할 클래스(카테고리)의 수를 설정합니다. 이 변수는 모델의 출력 레이어에 있는 뉴런의 수와 관련이 있습니다.

batch_size: 학습 중에 모델에 전달되는 이미지 배치의 크기를 설정합니다. 이는 데이터를 미니배치로 처리하는 데 사용됩니다.

num_workers: 데이터로더(DataLoader)에서 데이터를 로드하는 동안 사용되는 병렬 작업자(worker)의 수를 설정합니다. 병렬 작업자를 사용하면 데이터 로딩이 빨라지므로 학습이 빨라질 수 있습니다.

lr: 학습률(learning rate)은 모델의 가중치를 업데이트하는 속도를 제어하는 하이퍼파라미터입니다. 이 값은 경사 하강법(GD, Gradient Descent)의 한 부분으로 모델 학습을 조절하는 데 사용됩니다.

total_epoch: 전체 학습 에폭(epoch)의 수를 설정합니다. 에폭은 전체 데이터셋에 대한 학습 과정을 몇 번 반복할 것인지를 결정합니다.

device: 학습을 수행할 디바이스(GPU 또는 CPU)를 선택합니다. torch.cuda.is_available()를 사용하여 GPU가 사용 가능한 경우 GPU를 선택하고, 그렇지 않은 경우 CPU를 선택합니다. 딥러닝 모델 학습 시 GPU를 사용하면 학습 속도가 향상될 수 있습니다.

요약하면, 이 코드는 학습에 사용될 경로 및 하이퍼파라미터를 설정하고, 사용 가능한 GPU를 확인하여 디바이스를 선택하는 역할을 합니다. 이러한 설정은 모델 학습의 기본적인 구성 요소를 정의하며, 학습 작업을 시작하기 위한 필수 단계입니다.

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
import torchvision.datasets as datasets
import os


# Data loading code
traindir = os.path.join(dataset_path, 'train')
testdir = os.path.join(dataset_path, 'test')

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                   std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, pin_memory=True, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(testdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=num_workers, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


traindir 및 testdir: 학습 및 테스트 데이터가 저장된 디렉토리의 경로를 설정합니다. 이 경로는 데이터셋의 실제 위치를 지정합니다.

train_dataset: 학습 데이터를 로드합니다. datasets.ImageFolder를 사용하여 디렉토리 구조를 기반으로 이미지 파일을 로드하고, 데이터 증강 및 변환을 적용합니다. 이 변환에는 이미지를 임의 크기로 자르고, 수평으로 뒤집는 등의 작업이 포함될 수 있습니다.

train_loader: 학습 데이터를 로드하는 데이터 로더를 설정합니다. 데이터 로더는 미니배치를 생성하고, 데이터를 섞어서 모델의 학습을 돕는 역할을 합니다. batch_size는 미니배치 크기를, shuffle은 데이터를 섞을지 여부를, num_workers는 병렬 데이터 로딩을 수행할 작업자(worker) 수를 나타냅니다.

test_loader: 테스트 데이터를 로드하는 데이터 로더를 설정합니다. 테스트 데이터에는 데이터 증강 작업이 적용되지 않고, 이미지 크기 조정과 정규화만 수행됩니다. 마찬가지로 미니배치 크기와 병렬 데이터 로딩 작업자 수를 설정합니다.

이 코드는 데이터를 효율적으로 로드하고 전처리한 후, 모델 학습 및 평가를 위해 사용될 데이터 로더를 생성하는 역할을 합니다.

In [ ]:
model = SuperLightMobileNet(num_classes).to(device)

이 코드는 SuperLightMobileNet 클래스로부터 모델 객체를 생성하고, 해당 모델을 지정한 디바이스(GPU 또는 CPU)로 이동시키는 역할을 합니다.

In [ ]:
CEloss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

CEloss = nn.CrossEntropyLoss(): 이 줄은 크로스 엔트로피 손실 함수를 정의하고, 이를 CEloss 변수에 할당합니다. 크로스 엔트로피 손실 함수는 다중 클래스 분류 문제에서 주로 사용되며, 모델의 출력과 실제 레이블 간의 손실을 계산하는 데 사용됩니다. 모델이 분류 작업을 수행하면 손실 함수를 통해 모델의 성능을 평가하고 손실을 최소화하기 위해 모델 파라미터를 업데이트합니다.

optimizer = torch.optim.Adam(model.parameters(), lr=lr): 이 줄은 Adam 옵티마이저를 설정하고, 이를 optimizer 변수에 할당합니다. Adam은 경사 하강법의 한 변종으로, 모델 파라미터를 업데이트하기 위해 사용됩니다. model.parameters()는 모델의 학습 가능한 파라미터를 반환하며, lr은 학습률(learning rate)로, 가중치 업데이트 속도를 조절합니다. Adam 옵티마이저는 손실을 최소화하기 위해 모델 파라미터를 조정하는 역할을 합니다.

요약하면, 이 코드는 모델을 학습하기 위한 손실 함수와 옵티마이저를 설정하는 부분입니다. 크로스 엔트로피 손실 함수는 모델의 예측과 실제 레이블 간의 차이를 측정하고, Adam 옵티마이저는 손실을 최소화하기 위해 모델 가중치를 업데이트합니다. 이러한 구성 요소는 모델 학습 과정을 구성하는 중요한 부분입니다.






In [ ]:
import numpy as np

total_iteration_per_epoch = int(np.ceil(len(train_dataset)/batch_size))

for epoch in range(1, total_epoch + 1):
    model.train()
    for itereation, (input, target) in enumerate(train_loader):
        images = input.to(device)
        labels = target.to(device)

        # Forward pass
        outputs = model(images)
        loss = CEloss(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch [{}/{}], Iteration [{}/{}] Loss: {:.4f}'.format(epoch, total_epoch, itereation+1, total_iteration_per_epoch, loss.item()))
    if epoch % 10 == 0:
      torch.save(model.state_dict(), model_weight_save_path + 'models_' + str(epoch) + ".pth")

    model.eval()
    with torch.no_grad():
      correct = 0
      total = 0
      for input, target in test_loader:
          images = input.to(device)
          labels = target.to(device)

          # Forward pass
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += len(labels)
          correct += (predicted == labels).sum().item()

      print('Epoch [{}/{}], Test Accuracy of the model on the {} test images: {} %'.format(epoch, total_epoch, total, 100 * correct / total))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch [16/30], Iteration [86/338] Loss: 1.1912
Epoch [16/30], Iteration [87/338] Loss: 0.9494
Epoch [16/30], Iteration [88/338] Loss: 1.0198
Epoch [16/30], Iteration [89/338] Loss: 1.3423
Epoch [16/30], Iteration [90/338] Loss: 0.8149
Epoch [16/30], Iteration [91/338] Loss: 0.9712
Epoch [16/30], Iteration [92/338] Loss: 1.1645
Epoch [16/30], Iteration [93/338] Loss: 1.0500
Epoch [16/30], Iteration [94/338] Loss: 1.5109
Epoch [16/30], Iteration [95/338] Loss: 1.3777
Epoch [16/30], Iteration [96/338] Loss: 0.9257
Epoch [16/30], Iteration [97/338] Loss: 1.2130
Epoch [16/30], Iteration [98/338] Loss: 0.9482
Epoch [16/30], Iteration [99/338] Loss: 1.0276
Epoch [16/30], Iteration [100/338] Loss: 1.2414
Epoch [16/30], Iteration [101/338] Loss: 1.3236
Epoch [16/30], Iteration [102/338] Loss: 1.6145
Epoch [16/30], Iteration [103/338] Loss: 0.9384
Epoch [16/30], Iteration [104/338] Loss: 1.2349
Epoch [16/30], Iteration [105/338] Loss: 0.7492
Epoch [16/30], Ite

total_iteration_per_epoch: 각 에폭(epoch)에서 전체 학습 데이터를 미니배치로 처리하기 위한 총 반복 횟수를 계산합니다. len(train_dataset)은 학습 데이터셋의 전체 데이터 수이고, batch_size는 한 미니배치의 크기입니다. np.ceil을 사용하여 올림을 수행하고 정수로 변환합니다.

for epoch in range(1, total_epoch + 1): 지정된 총 에폭 수(total_epoch)에 대한 반복 루프를 시작합니다.

model.train(): 모델을 학습 모드로 설정합니다. 이렇게 하면 모델이 학습 중에는 드롭아웃(dropout) 레이어와 배치 정규화(batch normalization)를 학습 모드로 활성화합니다.

두 번째 for 루프: 이 루프는 학습 데이터를 미니배치로 나누고 모델을 학습하는 부분입니다. train_loader를 통해 미니배치 데이터를 가져오며, images는 입력 데이터, labels는 해당 미니배치의 타겟 레이블입니다.

Forward pass: 모델에 입력 데이터를 전달하여 출력을 얻습니다. 이후 이 출력을 사용하여 손실 함수(CEloss)를 계산합니다.

Backward and optimize: 역전파(backpropagation)를 수행하고 옵티마이저를 사용하여 모델 파라미터를 업데이트합니다.

print: 각 미니배치의 손실을 출력하여 학습 진행 상황을 모니터링합니다.

if epoch % 10 == 0: 매 10 에폭마다 모델의 가중치를 저장합니다. 이는 모델의 중간 상태를 저장하고 나중에 재학습 또는 모델 복원에 사용될 수 있습니다.

model.eval(): 모델을 평가 모드로 설정합니다. 이는 모델이 테스트 데이터에서는 드롭아웃과 배치 정규화를 평가 모드로 활성화합니다.

두 번째 for 루프(두 번째 루프 내의 내포된 루프): 이 루프는 테스트 데이터를 미니배치로 나누고 모델을 사용하여 정확도를 계산합니다.

torch.save: 특정 에폭(epoch)에서 모델의 가중치를 저장합니다. 모델 학습 중간 상태의 저장은 중요한 실험 및 결과의 재현성을 위해 사용됩니다.

print: 각 에폭에서 모델의 테스트 데이터 정확도를 출력하여 모델의 성능을 모니터링합니다.

이 코드는 모델을 학습하고 정기적으로 평가하여 학습 진행 상황을 모니터링하며, 모델의 가중치를 저장하는 중요한 학습 루프를 구현한 부분입니다.